In [13]:
from utils import count_nonzero_parameters, load_untrained_model

In [15]:
dict_ = load_untrained_model("DenseNet121")
model = dict_["model"]

In [16]:
count_nonzero_parameters(model)

6914538